In [1]:
import pandas as pd
import numpy as np
from functools import partial
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Question 1

1. Generate a report that tracks the various Genere combinations for each type year on year. The result
data frame should contain type, Genere_combo, year, avg_rating, min_rating, max_rating,
total_run_time_mins

In [2]:
imdb = pd.read_csv("imdb.csv", escapechar="\\")
imdb.head()

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,nrOfUserReviews,nrOfGenre,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,FilmNoir,GameShow,History,Horror,Music,Musical,Mystery,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,1,0,19,96,85,3,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,2,1,35,110,122,3,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,3,4,67,428,376,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,1,1,53,123,219,3,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,2,0,38,187,186,3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [3]:
def get_genre_combo(one_hot_vec, genres, join_with = " | "):
    target_genres = np.array(genres)[np.array(one_hot_vec, dtype=np.bool)]
    return join_with.join(target_genres)

In [4]:
genres_list = imdb.loc[:1, "Action":].columns
genres_combo_col = imdb.loc[:, "Action":].apply(partial(get_genre_combo, genres=genres_list), axis=1)

In [5]:
groups = imdb.groupby(["year", "type", genres_combo_col])
df_grouped = groups.agg({
    "imdbRating": [max, min, np.mean],
    "duration": lambda durations: sum(durations) / len(durations)
})

In [6]:
final_df = df_grouped.reset_index()
final_df.columns = ["year", "type", "genre_combo", "max_rating", "min_rating", "avg_rating", "total_run_time"]
final_df.sample(10)

,year,type,genre_combo,max_rating,min_rating,avg_rating,total_run_time
1538,1965.0,video.movie,Drama | Mystery | SciFi,7.3,7.3,7.3,5940.0
3292,1988.0,video.episode,Comedy,NaN,NaN,NaN,1260.0
4471,1997.0,video.movie,Animation | Horror | Mystery,7.8,7.8,7.8,4860.0
7271,2011.0,video.movie,Adventure | Comedy | Crime,6.3,6.3,6.3,6060.0
41,1917.0,video.movie,Adventure | Comedy | Drama,7.1,7.1,7.1,4200.0
3952,1993.0,video.movie,Documentary | History,7.4,7.4,7.4,5760.0
7408,2011.0,video.tv,TalkShow,NaN,NaN,NaN,NaN
3996,1993.0,video.tv,Drama | Mystery | SciFi,8.8,8.8,8.8,2700.0
5490,2003.0,video.movie,Adventure | Family | Fantasy,6.9,6.9,6.9,6780.0
5429,2003.0,video.episode,Action | Adventure | Animation,7.7,7.7,7.7,1500.0


## Question 2

Is there a realation between the length of a movie title and the ratings ? Generate a report that captures
the trend of the number of letters in movies titles over years. We expect a cross tab between the year of
the video release and the quantile that length fall under. The results should contain year, min_length,
max_length, num_videos_less_than25Percentile, num_videos_25_50Percentile ,
num_videos_50_75Percentile, num_videos_greaterthan75Precentile

In [7]:
def length(s):
    try:
        return len(s)
    except TypeError:
        return 1

In [8]:
year_groups = imdb.dropna(subset=["year", "wordsInTitle"]).groupby("year")
final_df = year_groups.agg({
    "wordsInTitle": [
        lambda x: min(x.apply(length)),
        lambda x: max(x.apply(length)),
        lambda x: len(x[x.apply(length) < np.percentile(x.apply(length), 25)]),
        lambda x: len(x[(x.apply(length) >= np.percentile(x.apply(length), 25)) & (x.apply(length) < np.percentile(x.apply(length), 50))]),
        lambda x: len(x[(x.apply(length) >= np.percentile(x.apply(length), 50)) & (x.apply(length) < np.percentile(x.apply(length), 75))]),
        lambda x: len(x[x.apply(length) >= np.percentile(x.apply(length), 75)]),
    ]
})

SpecificationError: Function names must be unique, found multiple named <lambda>

In [ ]:
final_df.columns = ["min_length",
"max_length", "num_videos_less_than25Percentile", "num_videos_25_50Percentile" ,
"num_videos_50_75Percentile", "num_videos_greaterthan75Precentile"]
final_df.reset_index(inplace=True)
final_df.sample(10)

## Question 3

In diamonds data set Using the volumne calculated above, create bins that have equal population within
them. Generate a report that contains cross tab between bins and cut. Represent the number under
each cell as a percentage of total.

**Note:** Since the number of groups is not mentioned, the data has been made into 11 groups, with each group containing 5000 items.

In [ ]:
diamonds = pd.read_csv("diamonds.csv", na_values={"z": ["None"]})
diamonds.head()

In [ ]:
diamond_volumes = diamonds. x * diamonds.y * diamonds.z

In [ ]:
bins = np.arange(len(diamonds)) // 5000
final_df = (diamonds.groupby([bins, diamonds.cut])
         .x
         .count()
         .unstack()
         .apply(lambda row: row / row.sum() * 100))
final_df

## Question 4

Generate a report that tracks the Avg. imdb rating quarter on quarter, in the last 10 years, for movies
that are top performing. You can take the top 10% grossing movies every quarter. Add the number of top
performing movies under each genere in the report as well.

**Note:** Since information is available for year only, this, instead of quarter, is considered.

In [11]:
movies = pd.read_csv("movie_metadata.csv")
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [12]:
latest_movies = movies[movies.title_year.isin(np.arange(2007, 2017))]
yearly_gross_cutoff = (latest_movies.dropna(subset=["title_year", "gross"])
                       .groupby("title_year")
                       .gross
                       .quantile(q=0.90))
yearly_gross_cutoff.name = "gross_cutoff"

top_movies = (latest_movies.merge(yearly_gross_cutoff.reset_index())
                    .query("gross >= gross_cutoff"))
del latest_movies

In [17]:
genres = set(top_movies.genres.str.cat(sep="|").split("|"))

In [23]:
for genre in genres:
    top_movies[genre] = top_movies.genres.apply(lambda x: genre in x)

In [37]:
final_df = (top_movies.groupby("title_year")
            .imdb_score.mean().reset_index()
            .merge(top_movies.loc[:, list(genres)].groupby(top_movies.title_year)
                   .sum().reset_index()))

,title_year,imdb_score,War,Sci-Fi,Biography,Sport,Crime,Musical,Animation,Comedy,Family,Adventure,Romance,Horror,Western,Action,Thriller,Drama,Music,History,Mystery,Fantasy
0,2007.0,6.833333,0.0,3.0,0.0,0.0,1.0,0.0,4.0,9.0,4.0,10.0,5.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,2.0,5.0
1,2008.0,6.990476,0.0,3.0,0.0,0.0,2.0,1.0,3.0,6.0,6.0,9.0,6.0,0.0,0.0,12.0,3.0,9.0,1.0,0.0,0.0,6.0
2,2009.0,6.804545,1.0,9.0,1.0,1.0,1.0,0.0,6.0,8.0,8.0,15.0,2.0,0.0,0.0,10.0,4.0,5.0,1.0,0.0,3.0,7.0
3,2010.0,6.800000,0.0,5.0,1.0,0.0,0.0,1.0,6.0,7.0,9.0,15.0,4.0,0.0,1.0,8.0,1.0,3.0,1.0,1.0,0.0,10.0
4,2011.0,6.870000,0.0,5.0,0.0,1.0,2.0,1.0,7.0,9.0,7.0,14.0,1.0,0.0,1.0,11.0,5.0,2.0,2.0,0.0,2.0,5.0
5,2012.0,7.252632,1.0,5.0,1.0,0.0,0.0,0.0,4.0,6.0,5.0,15.0,2.0,0.0,1.0,8.0,4.0,6.0,0.0,1.0,0.0,8.0
6,2013.0,7.242105,0.0,7.0,0.0,0.0,3.0,1.0,4.0,6.0,6.0,13.0,2.0,1.0,0.0,6.0,3.0,4.0,1.0,0.0,0.0,8.0
7,2014.0,7.264706,1.0,11.0,1.0,0.0,1.0,0.0,3.0,6.0,4.0,15.0,1.0,0.0,0.0,14.0,3.0,4.0,0.0,1.0,0.0,6.0
8,2015.0,7.141176,0.0,9.0,0.0,0.0,1.0,0.0,5.0,7.0,7.0,12.0,2.0,0.0,1.0,7.0,5.0,5.0,1.0,0.0,0.0,6.0
9,2016.0,7.475000,0.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,7.0,1.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,2.0
